##### Copyright 2022 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# RunInference with Sentenced T5 Model from TensorFlow Hub


In this notebook, we walk through the use of the RunInference transform with a [sentence encoder built on T5 model](https://tfhub.dev/google/sentence-t5/st5-base/1) and testing it locally with Interactive Runner.


## Install and import packages.

In [ ]:
!pip install apache_beam[gcp]==2.41.0
!pip install apache-beam[interactive]==2.41.0
!pip install tensorflow==2.10.0
!pip install tensorflow_text==2.10.0
!pip install keras==2.10.0
!pip install tfx_bsl==1.10.0
!pip install pillow==8.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 3.2 MB/s 
     |████████████████████████████████| 26.7 MB 98.6 MB/s 
     |████████████████████████████████| 278 kB 62.5 MB/s 
     |████████████████████████████████| 151 kB 61.0 MB/s 
     |████████████████████████████████| 62 kB 440 kB/s 
     |████████████████████████████████| 2.7 MB 22.2 MB/s 
     |████████████████████████████████| 526 kB 31.1 MB/s 
     |████████████████████████████████| 435 kB 25.2 MB/s 
     |████████████████████████████████| 125 kB 54.2 MB/s 
     |████████████████████████████████| 255 kB 36.7 MB/s 
     |████████████████████████████████| 236 kB 68.5 MB/s 
     |████████████████████████████████| 134 kB 51.3 MB/s 
     |████████████████████████████████| 183 kB 82.1 MB/s 
     |████████████████████████████████| 83 kB 744 kB/s 
     |████████████████████████████████| 271 kB 51.3 MB/s 
     |████████████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 7.1 MB/s 
     |████████████████████████████████| 186 kB 60.0 MB/s 
     |████████████████████████████████| 143 kB 53.5 MB/s 
     |████████████████████████████████| 423 kB 57.4 MB/s 
     |████████████████████████████████| 107 kB 63.1 MB/s 
     |████████████████████████████████| 1.6 MB 62.3 MB/s 
     |████████████████████████████████| 383 kB 69.6 MB/s 
  Created wheel for timeloop: filename=timeloop-1.0.2-py3-none-any.whl size=3720 sha256=3eccdf0399d44f59e9464837dfc5c5029560caca1efc8db4f4079a18508ee862
  Stored in directory: /root/.cache/pip/wheels/f8/69/2a/86476a2decfc2111dd1f10e723764a4d267b3bde7a47cd366e
Successfully built timeloop
  Attempting uninstall: traitlets
    Found existing installation: traitlets 5.1.1
    Uninstalling traitlets-5.1.1:
      Successfully uninstalled traitlets-5.1.1
  Attempting uninstall: tornado
    Found ex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.1 MB 7.5 kB/s 
     |████████████████████████████████| 438 kB 68.4 MB/s 
     |████████████████████████████████| 1.7 MB 44.3 MB/s 
     |████████████████████████████████| 5.9 MB 40.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully 

In [ ]:
import os
import importlib

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

from tensorflow import keras

from typing import Any
from typing import Dict
from typing import Iterable
from typing import Optional
from typing import Sequence

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib

from apache_beam.ml.inference.base import RunInference
from apache_beam.ml.inference.base import ModelHandler
from apache_beam.ml.inference.base import PredictionResult
from apache_beam.internal import pickler
from apache_beam.runners.runner import PipelineResult
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner

from tfx_bsl.public.beam.run_inference import CreateModelHandler
from tfx_bsl.public.proto import model_spec_pb2

2022-11-28 21:06:38.334716: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 21:06:38.619452: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-28 21:06:39.677456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-11-28 21:06:39.678648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

## Create a Keras Model from TensorFlow Hub Image

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
PROJECT_ID = '<Project Id>'
GCS_BUCKET = '<GCS Bucket>'

MODEL_PATH = f'{GCS_BUCKET}/st5-base/1'

In [ ]:
inp = tf.keras.layers.Input(shape=[], dtype=tf.string, name='input')
hub_url = "https://tfhub.dev/google/sentence-t5/st5-base/1"
imported = hub.KerasLayer(hub_url)
outp = imported(inp)
model = tf.keras.Model(inp, outp)

# Sentenced-T5 model returns a 768-dimensional vector for an English text input.
# Note the 'input' that we will pass in as example's feature key name.
model.summary()

2022-11-28 21:06:57.673124: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-28 21:06:57.673231: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c005624a5bf6): /proc/driver/nvidia/version does not exist
2022-11-28 21:06:57.674772: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None,)]                 0         
                                                                 
 keras_layer (KerasLayer)    [(None, 768)]             0         
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


## Save the model with a TF function definition for RunInference()

In [ ]:
RAW_DATA_PREDICT_SPEC = {
    'input': tf.io.FixedLenFeature([], tf.string),
}

@tf.function(input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string)])
def call(serialized_examples):
    features = tf.io.parse_example(serialized_examples, RAW_DATA_PREDICT_SPEC)
    return model(features)

tf.saved_model.save(model, MODEL_PATH, signatures={'serving_default': call})

## Create and test a RunInference pipeline locally

In [ ]:
# Creates TensorFlow Example to feed to the ModelHandler.
class ExampleProcessor:
    def create_example(self, feature: tf.string):
        return tf.train.Example(
            features=tf.train.Features(
                  feature={'input' : self.create_feature(feature)})
            )

    def create_feature(self, element: tf.string):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[element.encode()], ))


In [ ]:
saved_model_spec = model_spec_pb2.SavedModelSpec(model_path=MODEL_PATH)
inferece_spec_type = model_spec_pb2.InferenceSpecType(saved_model_spec=saved_model_spec)
model_handler = CreateModelHandler(inferece_spec_type)

questions = [
    'what is the official slogan for the 2018 winter olympics?',
]

pipeline = beam.Pipeline(InteractiveRunner())

inference = (pipeline | 'CreateSentences' >> beam.Create(questions)
               | 'Convert input to Tensor' >> beam.Map(lambda x: ExampleProcessor().create_example(x))
               | 'RunInference with T5' >> RunInference(model_handler))

In [ ]:
ib.show(inference)